In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [14]:
!pip install langdetect

     |████████████████████████████████| 1.0MB 4.7MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-cp36-none-any.whl size=993460 sha256=1bea9507e6d0cee774c8179482e3e1b34e2f4ae50a5a50f7c376c737eb6e146c
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
from __future__ import print_function


import json
import os
import pandas
import io
import sys
import re


class ProgressBar(object):
    DEFAULT = 'Progress: %(bar)s %(percent)3d%%'
    FULL = '%(bar)s %(current)d/%(total)d (%(percent)3d%%) %(remaining)d to go'

    def __init__(self, total, width=40, fmt=DEFAULT, symbol='=',
                 output=sys.stderr):
        assert len(symbol) == 1

        self.total = total
        self.width = width
        self.symbol = symbol
        self.output = output
        self.fmt = re.sub(r'(?P<name>%\(.+?\))d',
            r'\g<name>%dd' % len(str(total)), fmt)

        self.current = 0

    def __call__(self):
        percent = self.current / float(self.total)
        size = int(self.width * percent)
        remaining = self.total - self.current
        bar = '[' + self.symbol * size + ' ' * (self.width - size) + ']'

        args = {
            'total': self.total,
            'bar': bar,
            'current': self.current,
            'percent': percent * 100,
            'remaining': remaining
        }
        print('\r' + self.fmt % args, file=self.output, end='')

    def done(self):
        self.current = self.total
        self()
        print('', file=self.output)

In [0]:
import pandas as pd 

df1 = pd.read_csv("/content/drive/My Drive/Amhari/news___goolgule.csv") 
df2 = pd.read_csv("/content/drive/My Drive/Amhari/news___amhari___ethiopianregistrar.csv") 
df3 = pd.read_csv("/content/drive/My Drive/Amhari/news_amhari_ethsat.csv") 
df4 = pd.read_csv("/content/drive/My Drive/Amhari/news_amhari_ecadforum___.csv") 
df5 = pd.read_csv("/content/drive/My Drive/Amhari/news_zehabesha_archive.csv") 
df6 = pd.read_csv("/content/drive/My Drive/Amhari/news_amhari_selinum.csv") 
df7 = pd.read_csv("/content/drive/My Drive/Amhari/news_amhari_addisadmassnews.csv") 

frames = [df1, df2, df3 , df4 , df5 , df6 ,df7]
result = pd.concat(frames)

In [27]:
result.head()

,title,content,link,id
0,Test post,This is a test post for testing links and othe...,http://www.goolgule.com/2012/7/page/1/,652
1,title,content,link,id
2,ሰንደቅ፡ዓላማችንን,አረንጓዴ፡ብጫና፡ቀይ፡ሰንደቅ፡ዓላማችን፡ የአንድ፡ኢትዮጵያና፡ የነፃነት፡ ም...,http://www.goolgule.com/2012/9/page/1/,652
3,“ለዚህ (መንግሥት) ታላቅ ክብር አለኝ”,“ለኢትዮጵያ የጥምር መንግስት ያስፈልጋታል” በማለት በ1997 ምርጫ አጥብ...,http://www.goolgule.com/2012/9/page/1/,652
4,“ግራውንድ ሲቀነስ አንድ”,(ቀጭኑ ዘ-ቄራ )\nለአዲስ አበባ ከተማ ክብር ያላችሁ እንስማማለን። አዲ...,http://www.goolgule.com/2012/9/page/1/,652


In [4]:
len(result["title"])

74776

In [10]:
len(result.title.unique())

55335

In [0]:
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
!pip install fasttext
import fasttext
model = fasttext.load_model('lid.176.ftz')

In [42]:
print(model.predict('ዓላማችንን apple apple ዓላማችንን', k=2))  # top 2 matching languages

(('__label__am', '__label__cv'), array([0.90513647, 0.04793829]))


In [0]:
aa = model.predict('ዓላማችንን apple apple ዓላማችንን', k=2)

In [0]:
import string

special_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ»«'''
english_punctuations = string.punctuation
punctuations_list = special_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

def clean_text(text):
    #trim    
    text = text.strip()
    
    text = remove_punctuations(text)
    
    search =  ['\n'  , '\t', '&quot;' ,'?'  ,'؟'  ,'!'   ]
    replace = [' , ' , ' ' ,  ' '     ,' ? ',' ? ',' ! ']
        
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
           
    text = text.replace(".", " , ")

    # remove numbers
    text = ''.join([i for i in text if not i.isdigit()])
    return text

In [77]:
print(clean_text('ለዚህ (መንግሥት) ታላቅ ክብር አለኝ'))
print(clean_text('ለዚህ (መንግሥት) ታላቅ 7987 ክብር አለኝ'))

ለዚህ መንግሥት ታላቅ ክብር አለኝ
ለዚህ መንግሥት ታላቅ  ክብር አለኝ


In [78]:
df_clean = [] 
titles = [] 
progress = ProgressBar(len(result["title"]), fmt=ProgressBar.FULL)
for index, row in result.iterrows():
  row['title']   = clean_text(str(row['title']))
  row['content'] = clean_text(str(row['content']))
  if str(row['title']) != "nan" and str(row['title']) != "":
    row['title'] = row['title'].replace("\n" , " , ")
    detect = model.predict(row['title'], k=2)
    if detect[0][0] == "__label__am" and row['title'] not in titles:
      titles.append(row['title'])
      r={}
      r["title"] = row['title']
      r["content"] = row['content']
      df_clean.append(r)
    progress.current += 1
    progress()
progress.done()

[========================================] 74776/74776 (100%)     0 to go


In [79]:
print(len(df_clean))

53953


In [80]:
amhari_clean = pd.DataFrame(df_clean)
amhari_clean.head()

,title,content
0,ሰንደቅ፡ዓላማችንን,አረንጓዴ፡ብጫና፡ቀይ፡ሰንደቅ፡ዓላማችን፡ የአንድ፡ኢትዮጵያና፡ የነፃነት፡ ም...
1,ለዚህ መንግሥት ታላቅ ክብር አለኝ,ለኢትዮጵያ የጥምር መንግስት ያስፈልጋታል በማለት በ ምርጫ አጥብቆ ሲከራከ...
2,ግራውንድ ሲቀነስ አንድ,"ቀጭኑ ዘቄራ , ለአዲስ አበባ ከተማ ክብር ያላችሁ እንስማማለን። አዲስ ..."
3,ዶላር አዘዋዋሪዎቹ ደላሎች,ህወሓት የአጋር ፓርቲዎቹን የንግድ ተቋማትና ልማታዊ ባለሀብት እያለ የሚጠ...
4,ታላቅ ዕድል ነው,ዛሬ አርብ ልክ ከጠዋቱ ፡ ላይ የኢህአዴግ ወኪሎችና ኢህአዴግን እንደሚደ...


In [0]:
amhari_clean.to_csv("/content/drive/My Drive/Amhari/amhari_cleaner.csv", mode='w',index = False,encoding='utf-8')